# **Setup**


In [ ]:
pip install tensorflow keras opencv-python scikit-learn matplotlib nltk


## Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
nltk.download('punkt')


### Load and Preprocess Data

In [ ]:
def extract_frames(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in np.linspace(0, length-1, num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(i))
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (224, 224))
            frames.append(frame)
    cap.release()
    return np.array(frames)

def load_video_data(video_dir, labels_file, num_frames=10):
    labels = {}
    with open(labels_file, 'r') as file:
        for line in file.readlines():
            video, label = line.strip().split(',')
            labels[video] = int(label)

    X, y = [], []
    for video_file in os.listdir(video_dir):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(video_dir, video_file)
            frames = extract_frames(video_path, num_frames)
            X.append(frames)
            y.append(labels[video_file])

    return np.array(X), np.array(y)


#### Text Data Preprocessing

In [ ]:
def preprocess_text(text, vocab_size=5000, sequence_length=100):
    tokens = word_tokenize(text.lower())
    token_to_id = {token: idx+1 for idx, token in enumerate(tokens[:vocab_size-1])}
    sequences = [[token_to_id.get(token, 0) for token in tokens]]
    padded_sequences = pad_sequences(sequences, maxlen=sequence_length)
    return padded_sequences


### Define Models

#### NLP Model

In [ ]:
def build_nlp_model(vocab_size, embedding_dim, sequence_length):
    input_text = Input(shape=(sequence_length,))
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_text)
    lstm = LSTM(units=128, return_sequences=False)(embedding)
    dropout = Dropout(0.5)(lstm)
    output = Dense(1, activation='sigmoid')(dropout)
    model = Model(inputs=input_text, outputs=output)
    return model


#### Facial Recognition Model

In [ ]:
def build_facial_recognition_model(input_shape):
    input_img = Input(shape=input_shape)
    conv1 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(input_img)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    flatten = Flatten()(pool2)
    dense = Dense(units=128, activation='relu')(flatten)
    dropout = Dropout(0.5)(dense)
    output = Dense(1, activation='sigmoid')(dropout)
    model = Model(inputs=input_img, outputs=output)
    return model


### Integrate Models

#### Combined Model

In [ ]:
def build_combined_model(nlp_model, facial_model, sequence_length, input_shape):
    combined_input = Input(shape=(sequence_length + np.prod(input_shape),))
    
    # NLP branch
    nlp_input = tf.keras.layers.Lambda(lambda x: x[:, :sequence_length])(combined_input)
    nlp_output = nlp_model(nlp_input)
    
    # Facial recognition branch
    facial_input = tf.keras.layers.Lambda(lambda x: x[:, sequence_length:])(combined_input)
    facial_reshaped = tf.keras.layers.Reshape(input_shape)(facial_input)
    facial_output = facial_model(facial_reshaped)
    
    combined_output = tf.keras.layers.Concatenate()([nlp_output, facial_output])
    final_output = Dense(1, activation='sigmoid')(combined_output)
    
    combined_model = Model(inputs=combined_input, outputs=final_output)
    return combined_model


### Train and Evaluate

#### Training

In [ ]:
# Example hyperparameters
vocab_size = 5000
embedding_dim = 128
sequence_length = 100
input_shape = (224, 224, 3)

# Build models
nlp_model = build_nlp_model(vocab_size, embedding_dim, sequence_length)
facial_model = build_facial_recognition_model(input_shape)
combined_model = build_combined_model(nlp_model, facial_model, sequence_length, input_shape)

# Compile combined model
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load and preprocess data
# Note: Update paths to your dataset
video_dir = 'path_to_video_data'
labels_file = 'path_to_labels.csv'
X_videos, y = load_video_data(video_dir, labels_file)

# Assuming text data is preprocessed similarly
X_text = np.array([preprocess_text("example text data", vocab_size, sequence_length) for _ in range(len(y))])

# Combine video frames and text sequences
X_combined = np.concatenate([X_text.reshape(len(X_text), -1), X_videos.reshape(len(X_videos), -1)], axis=1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train the model
history = combined_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


#### Evaluation

In [ ]:
# Evaluate the model
loss, accuracy = combined_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


### Visualization

In [ ]:
# Plotting training & validation accuracy and loss
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_history(history)
